**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 2 - Open Science Platform - User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

In [1]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [2]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.

In [3]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-02-06 23:23:57,831 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-02-06 23:23:57.832 | INFO     | __main__:<module>:3 - b'3755751161cb4af7fba460f6fb38cce2b908d523bb33bafea1273ab73b613776'


2 - Creates a user account into the blockchain.

In [4]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_user_account(create_account_contract_address, user_account_short_id, DOMAIN, user_public_key, user_account)

2025-02-06 23:23:57.922 | INFO     | __main__:<module>:13 - User Role: author
2025-02-06 23:23:57.924 | INFO     | __main__:<module>:24 - Creating account with name: focused_beaver


2025-02-06 23:23:57.925 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-06 23:23:57.943 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'3755751161cb4af7fba460f6fb38cce2b908d523bb33bafea1273ab73b613776'
2025-02-06 23:23:57.944 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: BD0274D8D8C2EE62AEB41678578DA174A4CE7BA1
2025-02-06 23:23:57.946 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-06 23:23:57.947 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"
2025-02-06 23:23:57,969 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-06 23:23:57,971 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-06 23:23:59,196 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-06 23:23:59,205 - INFO - ('COMMITTED', 5, 0)
2025-02-06 23:23:59.209 | INFO     | dump_to_json:dump_to_j

b'e02b0e362aa3f6f59565eceb8cd7ae7f36b20d5d31e1b3f0c5615f323a093edc'

3 - Queries the blockchain to confirm the succesful creation of the account.

In [5]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-02-06 23:23:59.240 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-06 23:23:59,243 - WARNING - create_account_contract_address: BD0274D8D8C2EE62AEB41678578DA174A4CE7BA1
2025-02-06 23:23:59,260 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-06 23:23:59,262 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-06 23:24:02,233 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-06 23:24:02,242 - INFO - ('COMMITTED', 5, 0)
2025-02-06 23:24:02,244 - WARNING - Account Data: {'address': 'BD0274D8D8C2EE62AEB41678578DA174A4CE7BA1', 'hex_hash': '3e5761cdeb580808b20db5346ff0125f8c2c07bc34cb42d696be0856a3809c23'}
2025-02-06 23:24:02.245 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-06 23:24:02,251 - INFO - Appended data under 'focused_beaver@test' in logs/account_data.json
2025-02-06 23:24:02.252 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-06 23:24:02.254 | DEBUG    | integ

4 - Loads the smart contract for details (attributes) setting into the Iroha 1 blockchain .

In [6]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-06 23:24:02,296 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-06 23:24:02.297 | INFO     | __main__:<module>:3 - b'7a4c701789eeae86aef2ece85a451011078f426a726266ef6cca8bdfb381fb83'


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [7]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

2025-02-06 23:24:02.351 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-06 23:24:02.371 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'7a4c701789eeae86aef2ece85a451011078f426a726266ef6cca8bdfb381fb83'
2025-02-06 23:24:02.373 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 97556503B86187EE5F5797D47110770D61B5A362
2025-02-06 23:24:02.374 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-06 23:24:02.375 | DEBUG    | integration_helpers:tracer:35 - 	Entering "process_account"
2025-02-06 23:24:02,377 - INFO - Processing account: focused_beaver@test
2025-02-06 23:24:02,378 - INFO - User Account Metadata: {'@type': 'foaf:Person', 'foaf:name': 'Focused Beaver', 'foaf:mbox': 'focused_beaver@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Gulf University for Science and Techno

2025-02-06 23:24:02,410 - INFO - User Account Metadata CID: QmbDgw4zQZCLWEuuJKzuMcxfJYRcmzdBSJdhBt1thcxpCx
2025-02-06 23:24:02.412 | DEBUG    | integration_helpers:tracer:35 - 	Entering "set_account_detail"
2025-02-06 23:24:02,427 - INFO - None
2025-02-06 23:24:02,430 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-06 23:24:05,269 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-06 23:24:05,276 - INFO - ('COMMITTED', 5, 0)
2025-02-06 23:24:05.278 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-06 23:24:05.300 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001
2025-02-06 23:24:05.301 | DEBUG    | integration_helpers:get_engine_receipts_result:140 - Result decoded string:                                            

6 - Queries the blockchain again to confirm the succesfull details setting.

In [8]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid = process_raw_data(details)

2025-02-06 23:24:05.320 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-06 23:24:05,322 - WARNING - create_account_contract_address: BD0274D8D8C2EE62AEB41678578DA174A4CE7BA1
2025-02-06 23:24:05,338 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-06 23:24:05,340 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-06 23:24:08,296 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-06 23:24:08,303 - INFO - ('COMMITTED', 5, 0)
2025-02-06 23:24:08,306 - WARNING - Account Data: {'address': 'BD0274D8D8C2EE62AEB41678578DA174A4CE7BA1', 'hex_hash': '7c148a1253d21009160dd897ea381a0679dbc5d84aaf3e1e3eba31b48f77e775'}
2025-02-06 23:24:08.308 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-06 23:24:08,314 - INFO - Appended data under 'focused_beaver@test' in logs/account_data.json
2025-02-06 23:24:08.316 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-06 23:24:08.317 | DEBUG    | integ

7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [9]:
user_metadata = download_json_from_ipfs(account_metadata_cid)
logger.info(f"User Metadata: {user_metadata}")

2025-02-06 23:24:08.370 | INFO     | __main__:<module>:2 - User Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'focused_beaver@test', 'schema:publicKey': '341683d70e9e075df36032bba6ea54e41bb58770eb2c95b07d26cb00cd004eb2', 'schema:roleName': 'author'}, 'foaf:mbox': 'focused_beaver@email.com', 'foaf:name': 'Focused Beaver', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Gulf University for Science and Technology'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '1903-6335-5216-X'}}
